# UTILIDADES Y PLANTILLAS - LABORATORIO INTEGRADOR

## Contenido
Este cuaderno contiene **funciones de utilidad** y **plantillas de código** para usar en los laboratorios integradores.

### Propósito:
- **Acelerar el desarrollo** en los laboratorios
- **Evitar repetir código** común
- **Proporcionar ejemplos** de buenas prácticas
- **Facilitar la experimentación** rápida

---

## IMPORTACIONES ESTÁNDAR

In [ ]:
# ============================================
# IMPORTACIONES ESTÁNDAR - COPIAR Y PEGAR
# ============================================
# Copiar y pegar este bloque en cualquier laboratorio

# Procesamiento numérico y manipulación de datos
import numpy as np          # Operaciones numéricas y arrays multidimensionales
import pandas as pd         # Análisis y manipulación de datos estructurados

# Bibliotecas de visualización
import matplotlib.pyplot as plt    # Gráficos y visualizaciones básicas
import seaborn as sns             # Visualizaciones estadísticas avanzadas

# Procesamiento de imágenes - Bibliotecas principales
from PIL import Image, ImageEnhance           # Python Imaging Library - carga y manipulación básica
import cv2                                    # OpenCV - procesamiento avanzado (nota: usa BGR)
from skimage import color, feature, measure, filters, morphology, segmentation  # Scikit-image - algoritmos especializados

# Machine Learning y análisis predictivo
from sklearn.preprocessing import StandardScaler, LabelEncoder    # Preprocesamiento de datos
from sklearn.model_selection import train_test_split             # División de datasets
from sklearn.ensemble import RandomForestClassifier              # Algoritmo de clasificación
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix  # Métricas de evaluación

# Utilidades del sistema operativo
import os        # Navegación de archivos y carpetas
import glob      # Búsqueda de archivos con patrones
import warnings  # Control de mensajes de advertencia
warnings.filterwarnings('ignore')  # Suprimir advertencias para salida más limpia

# Específico para Google Colab
from google.colab import files    # Subida y descarga de archivos en Colab

# Configuración global de matplotlib para gráficos más grandes y legibles
plt.rcParams['figure.figsize'] = (12, 8)    # Tamaño por defecto de las figuras
plt.rcParams['font.size'] = 10              # Tamaño de fuente por defecto

# Semilla para reproducibilidad de resultados aleatorios
np.random.seed(42)  # Garantiza que los resultados aleatorios sean consistentes

print("✓ Importaciones cargadas correctamente")
print("✓ Configuración aplicada")
print("✓ Listo para comenzar el análisis")

## UTILIDADES DE IMÁGENES

In [ ]:
# =============================================
# UTILIDADES PARA MANEJO SEGURO DE IMÁGENES
# =============================================

def cargar_imagen_segura(path, target_size=None):
    """
    Carga una imagen de forma segura con manejo completo de errores.
    Convierte automáticamente a RGB y normaliza valores entre 0-1.

    Args:
        path (str): Ruta completa al archivo de imagen
        target_size (tuple): Tupla (ancho, alto) para redimensionar la imagen

    Returns:
        numpy.ndarray: Array normalizado (0-1) o None si hay error

    Ejemplo:
        img = cargar_imagen_segura('foto.jpg', target_size=(256, 256))
    """
    try:
        # Abrir imagen usando PIL (soporta múltiples formatos)
        img = Image.open(path)

        # Convertir a RGB si está en otro modo (RGBA, L, etc.)
        # Esto garantiza consistencia en el número de canales
        if img.mode != 'RGB':
            img = img.convert('RGB')

        # Redimensionar si se especifica un tamaño objetivo
        # LANCZOS ofrece mejor calidad para redimensionamiento
        if target_size:
            img = img.resize(target_size, Image.Resampling.LANCZOS)

        # Convertir a numpy array y normalizar a rango [0,1]
        # División por 255 convierte de uint8 [0,255] a float [0,1]
        return np.array(img) / 255.0

    except Exception as e:
        # Capturar cualquier error (archivo no encontrado, formato no válido, etc.)
        print(f"Error cargando {path}: {e}")
        return None

def mostrar_imagen_info(imagen, titulo="Imagen"):
    """
    Muestra información estadística completa de una imagen.
    Útil para debugging y análisis exploratorio.

    Args:
        imagen (numpy.ndarray): Array de la imagen
        titulo (str): Título descriptivo para la salida

    Ejemplo:
        mostrar_imagen_info(mi_imagen, "Imagen Original")
    """
    print(f"{titulo.upper()}:")
    print(f"   Shape: {imagen.shape}")                    # Dimensiones (altura, ancho, canales)
    print(f"   Tipo de datos: {imagen.dtype}")           # Tipo de datos (float64, uint8, etc.)
    print(f"   Rango de valores: [{imagen.min():.3f}, {imagen.max():.3f}]")  # Min y max
    print(f"   Valor promedio: {imagen.mean():.3f}")     # Media de todos los píxeles
    print(f"   Desviación estándar: {imagen.std():.3f}") # Variabilidad de los valores

def visualizar_canales_rgb(imagen, titulo="Análisis de Canales RGB"):
    """
    Visualiza los canales RGB de una imagen por separado.
    Útil para entender la contribución de cada color primario.

    Args:
        imagen (numpy.ndarray): Imagen RGB a analizar
        titulo (str): Título para la visualización

    Nota:
        Cada canal se muestra con su colormap correspondiente para mejor interpretación.
    """
    # Crear subplot con 4 columnas: original + 3 canales
    fig, axes = plt.subplots(1, 4, figsize=(16, 4))

    # Mostrar imagen original en color completo
    axes[0].imshow(imagen)
    axes[0].set_title('Imagen Completa (RGB)')
    axes[0].axis('off')  # Ocultar ejes para mejor visualización

    # Definir nombres y mapas de color para cada canal
    canales = ['Rojo (R)', 'Verde (G)', 'Azul (B)']
    colormaps = ['Reds', 'Greens', 'Blues']  # Mapas de color temáticos

    # Mostrar cada canal individual
    for i in range(3):
        # Extraer canal específico ([:,:,i] selecciona el canal i)
        axes[i+1].imshow(imagen[:,:,i], cmap=colormaps[i])
        axes[i+1].set_title(f'Canal {canales[i]}')
        axes[i+1].axis('off')

    # Configurar layout y mostrar
    plt.suptitle(titulo)
    plt.tight_layout()
    plt.show()

def convertir_espacios_color(imagen):
    """
    Convierte una imagen RGB a múltiples espacios de color.
    Útil para análisis comparativo y selección del espacio óptimo.

    Args:
        imagen (numpy.ndarray): Imagen RGB normalizada [0,1]

    Returns:
        dict: Diccionario con diferentes representaciones de la imagen

    Espacios incluidos:
        - RGB: Red, Green, Blue (original)
        - HSV: Hue (tono), Saturation (saturación), Value (valor)
        - Grayscale: Escala de grises (luminancia)
        - LAB: L*a*b* (perceptualmente uniforme)
    """
    espacios = {
        'rgb': imagen,                      # Espacio original
        'hsv': color.rgb2hsv(imagen),       # Separación tono/saturación/brillo
        'gray': color.rgb2gray(imagen),     # Información de luminancia únicamente
        'lab': color.rgb2lab(imagen)        # Espacio perceptualmente uniforme
    }

    return espacios

# Mensaje de confirmación para el usuario
print("✓ Utilidades de imágenes definidas y documentadas")
print("  → cargar_imagen_segura(): Carga robusta con manejo de errores")
print("  → mostrar_imagen_info(): Estadísticas detalladas de la imagen")
print("  → visualizar_canales_rgb(): Separación visual de canales")
print("  → convertir_espacios_color(): Múltiples representaciones")

## UTILIDADES DE SEGMENTACIÓN

In [ ]:
# =========================================================
# UTILIDADES AVANZADAS DE SEGMENTACIÓN DE IMÁGENES
# =========================================================

def segmentar_por_color_hsv_auto(imagen, num_colores=5):
    """
    Segmentación automática usando clustering K-means en espacio HSV.
    Agrupa píxeles similares por sus características de tono y saturación.

    Args:
        imagen (numpy.ndarray): Imagen RGB normalizada [0,1]
        num_colores (int): Número de clusters (regiones) deseadas

    Returns:
        tuple: (máscara_segmentada, modelo_kmeans)
        - máscara_segmentada: Array 2D con etiquetas de cluster por píxel
        - modelo_kmeans: Objeto KMeans entrenado para reutilización

    Ventaja del HSV: Separación natural entre tono (color) y brillo
    """
    from sklearn.cluster import KMeans

    # Convertir RGB a HSV para mejor separación de colores
    # HSV separa información cromática (H,S) de luminancia (V)
    imagen_hsv = color.rgb2hsv(imagen)

    # Obtener dimensiones para reshape posterior
    altura, ancho, _ = imagen_hsv.shape

    # Usar solo canales H (tono) y S (saturación) para clustering
    # El canal V (valor/brillo) puede variar por iluminación
    pixeles_hs = imagen_hsv[:,:,:2].reshape(-1, 2)  # Reshape a (n_pixeles, 2)

    # Aplicar K-means clustering
    # random_state=42 garantiza resultados reproducibles
    kmeans = KMeans(n_clusters=num_colores, random_state=42)
    etiquetas = kmeans.fit_predict(pixeles_hs)

    # Reformar etiquetas de vuelta a forma de imagen
    segmentacion = etiquetas.reshape(altura, ancho)

    return segmentacion, kmeans

def crear_mascara_color_rango(imagen_hsv, h_min, h_max, s_min=0.2, v_min=0.2):
    """
    Crea máscara binaria para segmentar un rango específico de colores.
    Útil para detectar objetos de colores conocidos (ej: frutas, señales).

    Args:
        imagen_hsv (numpy.ndarray): Imagen en espacio HSV [0,1]
        h_min, h_max (float): Rango de tono (Hue) [0,1]
        s_min (float): Saturación mínima (evita colores desaturados)
        v_min (float): Valor mínimo (evita píxeles muy oscuros)

    Returns:
        numpy.ndarray: Máscara binaria (1 donde cumple condiciones, 0 donde no)

    Ejemplo rangos de tono:
        - Rojo: 0.0-0.1 y 0.9-1.0 (el rojo está en los extremos)
        - Verde: 0.2-0.4
        - Azul: 0.5-0.7
    """
    # Crear máscaras individuales para cada condición
    mask_h = (imagen_hsv[:,:,0] >= h_min) & (imagen_hsv[:,:,0] <= h_max)  # Rango de tono
    mask_s = imagen_hsv[:,:,1] >= s_min  # Saturación mínima (evita grises)
    mask_v = imagen_hsv[:,:,2] >= v_min  # Valor mínimo (evita negro/sombras)

    # Combinar todas las condiciones con operador AND lógico
    # Un píxel debe cumplir TODAS las condiciones para ser incluido
    mask_final = mask_h & mask_s & mask_v

    # Convertir boolean a uint8 para compatibilidad con OpenCV
    return mask_final.astype(np.uint8)

def limpiar_mascara_morfologia(mascara, tam_kernel=3, operaciones=['close', 'open']):
    """
    Limpia máscaras binarias usando operaciones morfológicas.
    Elimina ruido y conecta regiones fragmentadas.

    Args:
        mascara (numpy.ndarray): Máscara binaria a limpiar
        tam_kernel (int): Tamaño del elemento estructurante (kernel)
        operaciones (list): Lista de operaciones a aplicar en orden

    Returns:
        numpy.ndarray: Máscara limpiada

    Operaciones disponibles:
        - 'close': Cierra huecos pequeños (dilatación + erosión)
        - 'open': Elimina ruido pequeño (erosión + dilatación)
        - 'erode': Reduce el tamaño de las regiones
        - 'dilate': Expande las regiones
    """
    from skimage.morphology import disk, closing, opening, erosion, dilation

    # Crear elemento estructurante circular
    # disk() crea un círculo que preserva mejor las formas naturales
    kernel = disk(tam_kernel)
    resultado = mascara.copy()

    # Aplicar operaciones en el orden especificado
    for op in operaciones:
        if op == 'close':
            # Closing: Conecta regiones cercanas y cierra huecos
            resultado = closing(resultado, kernel)
        elif op == 'open':
            # Opening: Elimina regiones pequeñas (ruido)
            resultado = opening(resultado, kernel)
        elif op == 'erode':
            # Erosión: Reduce tamaño de regiones (adelgaza bordes)
            resultado = erosion(resultado, kernel)
        elif op == 'dilate':
            # Dilatación: Expande regiones (engrosa bordes)
            resultado = dilation(resultado, kernel)

    return resultado

def analizar_regiones_conectadas(mascara_binaria, min_area=50):
    """
    Identifica y analiza objetos individuales en una máscara binaria.
    Extrae propiedades geométricas de cada región conectada.

    Args:
        mascara_binaria (numpy.ndarray): Máscara binaria con objetos
        min_area (int): Área mínima en píxeles para considerar un objeto

    Returns:
        list: Lista de objetos regionprops con propiedades geométricas

    Propiedades útiles de cada región:
        - .area: Área en píxeles
        - .bbox: Bounding box (min_row, min_col, max_row, max_col)
        - .centroid: Centro de masa
        - .eccentricity: Excentricidad (0=círculo, 1=línea)
        - .perimeter: Perímetro
    """
    # Etiquetar regiones conectadas (componentes conectados)
    # Cada objeto recibe una etiqueta numérica única
    etiquetas = measure.label(mascara_binaria)

    # Extraer propiedades geométricas de cada región
    # regionprops calcula múltiples características automáticamente
    regiones = measure.regionprops(etiquetas)

    # Filtrar regiones muy pequeñas (probablemente ruido)
    regiones_filtradas = [r for r in regiones if r.area >= min_area]

    return regiones_filtradas

def dibujar_bounding_boxes(ax, regiones, color='red', alpha=0.8):
    """
    Dibuja rectángulos delimitadores (bounding boxes) alrededor de objetos detectados.
    Útil para visualizar resultados de detección de objetos.

    Args:
        ax (matplotlib.axes): Eje donde dibujar los rectángulos
        regiones (list): Lista de objetos regionprops
        color (str): Color de los rectángulos
        alpha (float): Transparencia (0=transparente, 1=opaco)

    Nota: Debe llamarse después de ax.imshow() y antes de plt.show()
    """
    import matplotlib.patches as patches

    # Iterar sobre cada región detectada
    for region in regiones:
        # Obtener bounding box: (min_row, min_col, max_row, max_col)
        bbox = region.bbox

        # Crear rectángulo para matplotlib
        # Nota: matplotlib usa (x,y) = (col,row), por eso se invierten las coordenadas
        rect = patches.Rectangle(
            (bbox[1], bbox[0]),           # Esquina inferior izquierda (x,y)
            bbox[3] - bbox[1],            # Ancho (max_col - min_col)
            bbox[2] - bbox[0],            # Alto (max_row - min_row)
            linewidth=2,                  # Grosor del borde
            edgecolor=color,              # Color del borde
            facecolor='none',             # Sin relleno (solo borde)
            alpha=alpha                   # Transparencia
        )

        # Agregar rectángulo al eje
        ax.add_patch(rect)

# Mensajes informativos para el usuario
print("✓ Utilidades de segmentación definidas y documentadas")
print("  → segmentar_por_color_hsv_auto(): Clustering automático en HSV")
print("  → crear_mascara_color_rango(): Segmentación por rango de color")
print("  → limpiar_mascara_morfologia(): Limpieza morfológica de máscaras")
print("  → analizar_regiones_conectadas(): Detección de objetos individuales")
print("  → dibujar_bounding_boxes(): Visualización de detecciones")

## UTILIDADES DE EXTRACCIÓN DE CARACTERÍSTICAS

In [ ]:
# ==============================================================
# EXTRACCIÓN AVANZADA DE CARACTERÍSTICAS PARA MACHINE LEARNING
# ==============================================================

def extraer_estadisticas_basicas(imagen_canal):
    """
    Extrae estadísticas descriptivas fundamentales de un canal de imagen.
    Estas métricas capturan la distribución de intensidades en la imagen.

    Args:
        imagen_canal (numpy.ndarray): Canal individual de imagen (2D array)

    Returns:
        dict: Diccionario con estadísticas básicas

    Estadísticas calculadas:
        - Media: Intensidad promedio (indica brillo general)
        - Desviación estándar: Variabilidad de intensidades (indica contraste)
        - Mediana: Valor central (robusto a valores atípicos)
        - Cuartiles: Q25 y Q75 (rango intercuartílico)
        - Mínimo/Máximo: Rango completo de valores
    """
    # Aplanar imagen a vector 1D para cálculos estadísticos
    datos = imagen_canal.flatten()

    # Calcular estadísticas descriptivas
    estadisticas = {
        'media': np.mean(datos),              # Promedio aritmético
        'std': np.std(datos),                 # Desviación estándar (medida de dispersión)
        'mediana': np.median(datos),          # Valor del percentil 50
        'q25': np.percentile(datos, 25),      # Primer cuartil
        'q75': np.percentile(datos, 75),      # Tercer cuartil
        'min': np.min(datos),                 # Valor mínimo
        'max': np.max(datos)                  # Valor máximo
    }

    return estadisticas

def extraer_features_color_completas(imagen):
    """
    Extrae características completas de color en espacios RGB y HSV.
    Combina información cromática de ambos espacios para mayor robustez.

    Args:
        imagen (numpy.ndarray): Imagen RGB normalizada [0,1]

    Returns:
        numpy.ndarray: Vector de características de color (15 elementos)

    Características extraídas:
        RGB: 9 features (media, std, mediana por canal R, G, B)
        HSV: 6 features (media, std por canal H, S, V)
        Total: 15 características de color
    """
    features = []

    # CARACTERÍSTICAS RGB (3 canales × 3 estadísticas = 9 features)
    for canal in range(3):
        stats = extraer_estadisticas_basicas(imagen[:,:,canal])
        # Seleccionar las 3 estadísticas más informativas por canal
        features.extend([stats['media'], stats['std'], stats['mediana']])

    # CARACTERÍSTICAS HSV (separación cromática/luminancia)
    imagen_hsv = color.rgb2hsv(imagen)
    for canal in range(3):
        stats = extraer_estadisticas_basicas(imagen_hsv[:,:,canal])
        # HSV: solo media y std son más relevantes (mediana menos informativa)
        features.extend([stats['media'], stats['std']])

    return np.array(features)

def extraer_features_textura_avanzadas(imagen):
    """
    Extrae características avanzadas de textura usando filtros de gradiente.
    Captura información sobre patrones, bordes y rugosidad de la superficie.

    Args:
        imagen (numpy.ndarray): Imagen RGB o escala de grises

    Returns:
        numpy.ndarray: Vector de características de textura (7 elementos)

    Técnicas utilizadas:
        - Filtros de Sobel: Detectan gradientes horizontales y verticales
        - Detector de Canny: Identifica bordes bien definidos
        - Estadísticas de intensidad: Propiedades básicas de luminancia
    """
    # Convertir a escala de grises si es necesario (textura no necesita color)
    if len(imagen.shape) == 3:
        imagen_gray = color.rgb2gray(imagen)
    else:
        imagen_gray = imagen

    features = []

    # FILTROS DE SOBEL para detección de gradientes
    # Sobel horizontal: detecta cambios verticales (líneas horizontales)
    sobel_h = filters.sobel_h(imagen_gray)
    # Sobel vertical: detecta cambios horizontales (líneas verticales)
    sobel_v = filters.sobel_v(imagen_gray)

    # Estadísticas de gradientes (intensidad y variabilidad de bordes)
    features.extend([
        np.mean(np.abs(sobel_h)),    # Intensidad promedio de gradiente horizontal
        np.mean(np.abs(sobel_v)),    # Intensidad promedio de gradiente vertical
        np.std(sobel_h),             # Variabilidad de gradiente horizontal
        np.std(sobel_v)              # Variabilidad de gradiente vertical
    ])

    # DETECTOR DE CANNY para bordes bien definidos
    # Canny es más selectivo que Sobel (menos ruido, bordes más limpios)
    bordes = feature.canny(imagen_gray)
    # Densidad de bordes: proporción de píxeles que son borde
    features.append(np.sum(bordes) / bordes.size)

    # CARACTERÍSTICAS DE INTENSIDAD básicas
    features.extend([
        np.mean(imagen_gray),        # Brillo promedio
        np.std(imagen_gray)          # Contraste general
    ])

    return np.array(features)

def calcular_histograma_color(imagen, bins=32):
    """
    Calcula histograma concatenado de los 3 canales RGB.
    Los histogramas capturan la distribución de colores en la imagen.

    Args:
        imagen (numpy.ndarray): Imagen RGB [0,1]
        bins (int): Número de bins por canal (resolución del histograma)

    Returns:
        numpy.ndarray: Histograma concatenado normalizado (bins×3 elementos)

    Normalización: Cada histograma suma 1.0 (distribución de probabilidad)
    Ventaja: Invariante al tamaño de la imagen
    """
    histograma = []

    # Procesar cada canal RGB por separado
    for canal in range(3):
        # Calcular histograma en el rango [0,1] (imágenes normalizadas)
        hist, _ = np.histogram(imagen[:,:,canal], bins=bins, range=(0, 1))

        # Normalizar para que sume 1 (convertir a probabilidades)
        hist = hist / np.sum(hist)
        histograma.extend(hist)

    return np.array(histograma)

def extraer_features_completas(imagen):
    """
    Combina TODAS las características disponibles en un vector único.
    Esta función integra color, textura e histograma para máxima información.

    Args:
        imagen (numpy.ndarray): Imagen RGB normalizada [0,1]

    Returns:
        numpy.ndarray: Vector completo de características

    Composición del vector:
        - Características de color: 15 elementos (RGB + HSV)
        - Características de textura: 7 elementos (gradientes + bordes)
        - Histograma reducido: 24 elementos (8 bins × 3 canales)
        Total: 46 características por imagen
    """
    features = []

    # BLOQUE 1: Características de color (RGB + HSV)
    features_color = extraer_features_color_completas(imagen)
    features.extend(features_color)

    # BLOQUE 2: Características de textura (gradientes y bordes)
    features_textura = extraer_features_textura_avanzadas(imagen)
    features.extend(features_textura)

    # BLOQUE 3: Histograma compacto (reducido para evitar sobreajuste)
    # 8 bins por canal = 24 features totales (balance información/dimensionalidad)
    histograma = calcular_histograma_color(imagen, bins=8)
    features.extend(histograma)

    return np.array(features)

# Mensajes informativos para el usuario
print("✓ Utilidades de extracción de características definidas y documentadas")
print("  → extraer_estadisticas_basicas(): Estadísticas descriptivas por canal")
print("  → extraer_features_color_completas(): Características RGB + HSV")
print("  → extraer_features_textura_avanzadas(): Gradientes y detección de bordes")
print("  → calcular_histograma_color(): Histogramas normalizados por canal")
print("  → extraer_features_completas(): Vector integrado de 46 características")

## UTILIDADES DE MACHINE LEARNING

In [ ]:
# ===========================================================
# UTILIDADES COMPLETAS PARA MACHINE LEARNING EN IMÁGENES
# ===========================================================

def preparar_datos_ml(X, y, test_size=0.2, val_size=0.2):
    """
    Prepara datos para entrenamiento de ML con división estratificada en train/val/test.
    Incluye codificación de etiquetas y normalización de características.

    Args:
        X (array-like): Matriz de características (n_muestras, n_features)
        y (array-like): Vector de etiquetas (puede ser string o numérico)
        test_size (float): Proporción para conjunto de prueba (0.2 = 20%)
        val_size (float): Proporción de train para validación (0.2 = 20% del 80% restante)

    Returns:
        dict: Diccionario con conjuntos preparados y objetos de preprocesamiento

    Conjuntos generados:
        - Train: 60% de datos (para entrenar modelo)
        - Validation: 20% de datos (para ajuste de hiperparámetros)
        - Test: 20% de datos (para evaluación final)
    """
    # CODIFICACIÓN DE ETIQUETAS (si son texto)
    # Convertir categorías string a números enteros
    if isinstance(y[0], str):
        label_encoder = LabelEncoder()
        y_encoded = label_encoder.fit_transform(y)
        print(f"✓ Etiquetas codificadas: {list(label_encoder.classes_)}")
    else:
        y_encoded = y
        label_encoder = None
        print("✓ Etiquetas numéricas detectadas (sin codificación)")

    # PRIMERA DIVISIÓN: Train (80%) + Test (20%)
    # stratify=y_encoded mantiene proporción de clases en ambos conjuntos
    X_train, X_test, y_train, y_test = train_test_split(
        X, y_encoded,
        test_size=test_size,        # 20% para test
        random_state=42,           # Reproducibilidad
        stratify=y_encoded         # Mantener distribución de clases
    )

    # SEGUNDA DIVISIÓN: Train (60%) + Validation (20%)
    # val_size se aplica sobre el conjunto de entrenamiento temporal
    X_train_final, X_val, y_train_final, y_val = train_test_split(
        X_train, y_train,
        test_size=val_size,        # 20% del 80% restante = 16% total → ajustado a 25% para obtener 20% final
        random_state=42,
        stratify=y_train
    )

    # NORMALIZACIÓN DE CARACTERÍSTICAS
    # StandardScaler: media=0, desviación=1 para cada característica
    # Importante: fit solo en train, transform en todos los conjuntos
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_final)  # Fit + transform en train
    X_val_scaled = scaler.transform(X_val)                # Solo transform en val
    X_test_scaled = scaler.transform(X_test)              # Solo transform en test

    # PREPARAR RESULTADO
    resultado = {
        # Conjuntos de datos normalizados
        'X_train': X_train_scaled,
        'X_val': X_val_scaled,
        'X_test': X_test_scaled,
        # Etiquetas correspondientes
        'y_train': y_train_final,
        'y_val': y_val,
        'y_test': y_test,
        # Objetos de preprocesamiento (para nuevas predicciones)
        'scaler': scaler,
        'label_encoder': label_encoder
    }

    # REPORTE DE DIMENSIONES
    print(f"\n✓ Conjuntos de datos preparados:")
    print(f"   • Train: {X_train_scaled.shape[0]} muestras ({X_train_scaled.shape[0]/len(X)*100:.1f}%)")
    print(f"   • Validation: {X_val_scaled.shape[0]} muestras ({X_val_scaled.shape[0]/len(X)*100:.1f}%)")
    print(f"   • Test: {X_test_scaled.shape[0]} muestras ({X_test_scaled.shape[0]/len(X)*100:.1f}%)")
    print(f"   • Características: {X_train_scaled.shape[1]} features por muestra")

    return resultado

def entrenar_modelo_rapido(datos, modelo=None):
    """
    Entrena un modelo de clasificación con evaluación automática.
    Útil para prototipos rápidos y validación de características.

    Args:
        datos (dict): Diccionario retornado por preparar_datos_ml()
        modelo (sklearn.base.BaseEstimator): Modelo a entrenar (opcional)

    Returns:
        tuple: (modelo_entrenado, predicciones_val, predicciones_test)

    Modelo por defecto: RandomForest (robusto, interpretable, buen rendimiento)
    """
    # Usar RandomForest si no se especifica modelo
    # RF es robusto a overfitting y funciona bien sin mucho tuning
    if modelo is None:
        modelo = RandomForestClassifier(
            n_estimators=100,      # 100 árboles (balance velocidad/rendimiento)
            random_state=42,       # Reproducibilidad
            max_depth=10,          # Limitar profundidad (evitar overfitting)
            min_samples_split=5    # Mínimo de muestras para dividir nodo
        )
        print("✓ Usando RandomForest por defecto")
    else:
        print(f"✓ Usando modelo personalizado: {type(modelo).__name__}")

    # ENTRENAMIENTO
    print("⏳ Entrenando modelo...")
    modelo.fit(datos['X_train'], datos['y_train'])

    # PREDICCIONES
    # Validación: para ajuste de hiperparámetros
    y_pred_val = modelo.predict(datos['X_val'])
    # Test: para evaluación final (solo se usa una vez)
    y_pred_test = modelo.predict(datos['X_test'])

    # MÉTRICAS DE RENDIMIENTO
    acc_val = accuracy_score(datos['y_val'], y_pred_val)
    acc_test = accuracy_score(datos['y_test'], y_pred_test)

    print(f"\n✓ Entrenamiento completado:")
    print(f"   • Accuracy en Validación: {acc_val:.3f} ({acc_val*100:.1f}%)")
    print(f"   • Accuracy en Test: {acc_test:.3f} ({acc_test*100:.1f}%)")

    # Interpretación automática de resultados
    if acc_val > 0.9:
        print("   → Excelente rendimiento")
    elif acc_val > 0.8:
        print("   → Buen rendimiento")
    elif acc_val > 0.7:
        print("   → Rendimiento aceptable")
    else:
        print("   → Rendimiento bajo - revisar características o modelo")

    # Warning sobre overfitting
    if (acc_val - acc_test) > 0.1:
        print("   ⚠️  Posible overfitting (gran diferencia val-test)")

    return modelo, y_pred_val, y_pred_test

def mostrar_matriz_confusion(y_true, y_pred, labels=None, title="Matriz de Confusión"):
    """
    Visualiza matriz de confusión con formato profesional.
    Permite identificar errores de clasificación por clase.

    Args:
        y_true (array-like): Etiquetas reales
        y_pred (array-like): Etiquetas predichas
        labels (list): Nombres de clases para ejes (opcional)
        title (str): Título del gráfico

    La matriz muestra:
        - Diagonal: Clasificaciones correctas
        - Fuera de diagonal: Errores de clasificación
    """
    # Calcular matriz de confusión
    cm = confusion_matrix(y_true, y_pred)

    # Crear visualización con seaborn (más elegante que matplotlib básico)
    plt.figure(figsize=(8, 6))

    # Heatmap con anotaciones numéricas
    sns.heatmap(cm,
                annot=True,           # Mostrar números en celdas
                fmt='d',              # Formato entero
                cmap='Blues',         # Escala de colores azul
                xticklabels=labels,   # Etiquetas eje X
                yticklabels=labels,   # Etiquetas eje Y
                cbar_kws={'label': 'Número de muestras'})

    # Configuración de ejes y título
    plt.title(title, fontsize=14, fontweight='bold')
    plt.xlabel('Predicción del Modelo', fontsize=12)
    plt.ylabel('Valor Real', fontsize=12)
    plt.tight_layout()
    plt.show()

    # Calcular métricas adicionales de la matriz
    total_muestras = np.sum(cm)
    aciertos = np.trace(cm)  # Suma de la diagonal
    print(f"\n📊 Análisis de la matriz:")
    print(f"   • Total de muestras: {total_muestras}")
    print(f"   • Aciertos totales: {aciertos}")
    print(f"   • Errores totales: {total_muestras - aciertos}")

def mostrar_importancia_features(modelo, feature_names=None, top_n=10):
    """
    Visualiza importancia de características para modelos basados en árboles.
    Ayuda a entender qué características son más relevantes para la clasificación.

    Args:
        modelo: Modelo entrenado con atributo feature_importances_
        feature_names (list): Nombres de características (opcional)
        top_n (int): Número de características más importantes a mostrar

    Solo funciona con: RandomForest, ExtraTrees, GradientBoosting, etc.
    """
    # Verificar si el modelo soporta importancia de características
    if hasattr(modelo, 'feature_importances_'):
        importancias = modelo.feature_importances_

        # Obtener índices de las características más importantes
        indices = np.argsort(importancias)[-top_n:]  # Top N (orden ascendente)

        # Crear nombres por defecto si no se proporcionan
        if feature_names is None:
            feature_names = [f'Feature_{i}' for i in range(len(importancias))]

        # Crear gráfico horizontal (mejor para nombres largos)
        plt.figure(figsize=(10, 6))
        plt.barh(range(len(indices)), importancias[indices], color='skyblue', alpha=0.8)

        # Configurar ejes
        plt.yticks(range(len(indices)), [feature_names[i] for i in indices])
        plt.xlabel('Importancia Relativa', fontsize=12)
        plt.title(f'Top {top_n} Características Más Importantes', fontsize=14, fontweight='bold')

        # Agregar valores en las barras
        for i, v in enumerate(importancias[indices]):
            plt.text(v + 0.001, i, f'{v:.3f}', va='center', fontsize=10)

        plt.grid(axis='x', alpha=0.3)
        plt.tight_layout()
        plt.show()

        # Mostrar valores numéricos también
        print(f"\n🔍 Top {top_n} características más importantes:")
        for i, idx in enumerate(reversed(indices)):  # Orden descendente
            print(f"   {i+1:2d}. {feature_names[idx]:<30} {importancias[idx]:.4f}")

    else:
        print(f"❌ El modelo {type(modelo).__name__} no soporta importancia de características")
        print("   Modelos compatibles: RandomForest, ExtraTrees, GradientBoosting")

# Mensajes informativos para el usuario
print("✓ Utilidades de Machine Learning definidas y documentadas")
print("  → preparar_datos_ml(): División estratificada train/val/test + normalización")
print("  → entrenar_modelo_rapido(): Entrenamiento con evaluación automática")
print("  → mostrar_matriz_confusion(): Visualización profesional de errores")
print("  → mostrar_importancia_features(): Análisis de relevancia de características")

## UTILIDADES DE VISUALIZACIÓN

In [ ]:
# ====================================================
# UTILIDADES AVANZADAS DE VISUALIZACIÓN
# ====================================================

def crear_grid_imagenes(imagenes, titulos=None, filas=2, figsize=(15, 8)):
    """
    Crea un grid organizado de imágenes para comparación visual.
    Útil para mostrar múltiples imágenes en una sola figura.

    Args:
        imagenes (list): Lista de arrays de imágenes
        titulos (list): Lista de títulos para cada imagen (opcional)
        filas (int): Número de filas en el grid
        figsize (tuple): Tamaño de la figura (ancho, alto)

    Maneja automáticamente:
        - Imágenes en escala de grises (2D arrays)
        - Imágenes en color (3D arrays)
        - Grids irregulares (espacios vacíos)
    """
    n_imagenes = len(imagenes)
    # Calcular columnas necesarias basado en número de imágenes y filas
    columnas = int(np.ceil(n_imagenes / filas))

    # Crear subplot grid
    fig, axes = plt.subplots(filas, columnas, figsize=figsize)

    # Manejar caso especial de una sola imagen
    if n_imagenes == 1:
        axes = [axes]
    elif filas == 1:
        axes = axes.reshape(1, -1)
    elif columnas == 1:
        axes = axes.reshape(-1, 1)

    # Aplanar array de axes para fácil iteración
    axes = axes.flatten()

    # Mostrar cada imagen
    for i, img in enumerate(imagenes):
        # Detectar si es escala de grises o color
        if len(img.shape) == 2:  # Escala de grises
            axes[i].imshow(img, cmap='gray')
        else:  # Color (RGB)
            axes[i].imshow(img)

        # Agregar título si se proporciona
        if titulos and i < len(titulos):
            axes[i].set_title(titulos[i], fontsize=12)

        # Ocultar ejes para mejor visualización
        axes[i].axis('off')

    # Ocultar axes sobrantes en caso de grid irregular
    for i in range(n_imagenes, len(axes)):
        axes[i].axis('off')

    plt.tight_layout()
    plt.show()

def mostrar_antes_despues(img_original, img_procesada, titulo_original="Original", titulo_procesada="Procesada"):
    """
    Muestra comparación lado a lado de imagen original vs procesada.
    Perfecto para visualizar el efecto de operaciones de procesamiento.

    Args:
        img_original (numpy.ndarray): Imagen antes del procesamiento
        img_procesada (numpy.ndarray): Imagen después del procesamiento
        titulo_original (str): Título para imagen original
        titulo_procesada (str): Título para imagen procesada
    """
    # Crear subplot con 2 columnas
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

    # Mostrar imagen original
    ax1.imshow(img_original)
    ax1.set_title(titulo_original, fontsize=14, fontweight='bold')
    ax1.axis('off')

    # Mostrar imagen procesada (detectar si es escala de grises)
    if len(img_procesada.shape) == 2:
        ax2.imshow(img_procesada, cmap='gray')
    else:
        ax2.imshow(img_procesada)
    ax2.set_title(titulo_procesada, fontsize=14, fontweight='bold')
    ax2.axis('off')

    plt.tight_layout()
    plt.show()

def crear_dashboard_imagen(imagen, mostrar_canales=True, mostrar_histograma=True, mostrar_estadisticas=True):
    """
    Crea un dashboard completo de análisis visual de imagen.
    Combina múltiples visualizaciones en una sola figura integral.

    Args:
        imagen (numpy.ndarray): Imagen RGB a analizar
        mostrar_canales (bool): Mostrar separación de canales RGB
        mostrar_histograma (bool): Mostrar histograma de intensidades
        mostrar_estadisticas (bool): Mostrar panel de estadísticas

    Genera:
        - Imagen original
        - Canales RGB separados
        - Histograma de distribución de colores
        - Panel de estadísticas descriptivas
    """
    # Crear figura grande con subplot grid
    fig = plt.figure(figsize=(16, 12))

    # Imagen original (posición principal)
    ax1 = plt.subplot(3, 3, 1)
    plt.imshow(imagen)
    plt.title('Imagen Original', fontsize=14, fontweight='bold')
    plt.axis('off')

    if mostrar_canales and len(imagen.shape) == 3:
        # Canales RGB individuales
        canales = ['Rojo', 'Verde', 'Azul']
        cmaps = ['Reds', 'Greens', 'Blues']

        for i in range(3):
            ax = plt.subplot(3, 3, i + 2)
            plt.imshow(imagen[:,:,i], cmap=cmaps[i])
            plt.title(f'Canal {canales[i]}', fontsize=12)
            plt.axis('off')

    if mostrar_histograma:
        # Histograma de distribución de colores
        ax5 = plt.subplot(3, 3, 5)
        if len(imagen.shape) == 3:
            colors = ['red', 'green', 'blue']
            # Histograma superpuesto para cada canal
            for i in range(3):
                plt.hist(imagen[:,:,i].flatten(), bins=50, alpha=0.6,
                        color=colors[i], label=canales[i])
            plt.legend()
        else:
            plt.hist(imagen.flatten(), bins=50, alpha=0.7, color='gray')

        plt.title('Histograma de Intensidades', fontsize=12)
        plt.xlabel('Intensidad')
        plt.ylabel('Frecuencia')
        plt.grid(True, alpha=0.3)

    if mostrar_estadisticas:
        # Panel de estadísticas descriptivas
        ax6 = plt.subplot(3, 3, 6)
        ax6.axis('off')  # Sin ejes para texto puro

        # Compilar texto estadístico
        stats_text = "📊 ESTADÍSTICAS:\n\n"
        stats_text += f"📐 Dimensiones: {imagen.shape}\n"
        stats_text += f"🔢 Tipo de datos: {imagen.dtype}\n"
        stats_text += f"📉 Rango: [{imagen.min():.3f}, {imagen.max():.3f}]\n"
        stats_text += f"📊 Media global: {imagen.mean():.3f}\n"
        stats_text += f"📈 Desv. estándar: {imagen.std():.3f}\n\n"

        # Estadísticas por canal si es color
        if len(imagen.shape) == 3:
            stats_text += "🎨 POR CANAL:\n"
            canales_nombres = ['🔴 Rojo', '🟢 Verde', '🔵 Azul']
            for i in range(3):
                canal_mean = imagen[:,:,i].mean()
                canal_std = imagen[:,:,i].std()
                stats_text += f"{canales_nombres[i]}: μ={canal_mean:.3f}, σ={canal_std:.3f}\n"

        # Mostrar texto con formato monoespaciado
        plt.text(0.05, 0.95, stats_text, fontsize=10,
                verticalalignment='top', fontfamily='monospace',
                bbox=dict(boxstyle="round,pad=0.3", facecolor="lightgray", alpha=0.8))

    plt.suptitle('Dashboard de Análisis de Imagen', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

# Mensajes informativos para el usuario
print("✓ Utilidades de visualización definidas y documentadas")
print("  → crear_grid_imagenes(): Grid organizado para múltiples imágenes")
print("  → mostrar_antes_despues(): Comparación lado a lado")
print("  → crear_dashboard_imagen(): Dashboard integral de análisis")

## PLANTILLAS RÁPIDAS

In [ ]:
# =============================================
# PLANTILLAS RÁPIDAS PARA TAREAS COMUNES
# =============================================

def analisis_rapido_imagen(path_imagen):
    """
    PLANTILLA 1: Análisis completo y rápido de una imagen individual.
    Combina carga, análisis estadístico, visualización y conversión de espacios.

    Args:
        path_imagen (str): Ruta al archivo de imagen

    Returns:
        tuple: (imagen_cargada, dict_espacios_color) o None si hay error

    Procesos incluidos:
        - Carga segura con redimensionamiento estándar
        - Análisis estadístico completo
        - Dashboard visual integrado
        - Conversión a múltiples espacios de color
        - Visualización comparativa de espacios
    """
    print("🚀 Iniciando análisis rápido de imagen...")

    # PASO 1: Cargar imagen con tamaño estándar para análisis
    imagen = cargar_imagen_segura(path_imagen, target_size=(256, 256))
    if imagen is None:
        print("❌ Error: No se pudo cargar la imagen")
        return None

    print("✓ Imagen cargada correctamente")

    # PASO 2: Análisis estadístico básico
    print("\n📊 Información estadística:")
    mostrar_imagen_info(imagen, "Análisis Rápido")

    # PASO 3: Dashboard visual completo
    print("\n📈 Generando dashboard visual...")
    crear_dashboard_imagen(imagen)

    # PASO 4: Conversión a múltiples espacios de color
    print("\n🎨 Convirtiendo espacios de color...")
    espacios = convertir_espacios_color(imagen)

    # PASO 5: Visualización comparativa de espacios
    imagenes_espacios = [espacios['rgb'], espacios['hsv'], espacios['gray']]
    titulos_espacios = ['RGB (Original)', 'HSV (Tono-Saturación)', 'Escala de Grises']
    crear_grid_imagenes(imagenes_espacios, titulos_espacios, filas=1)

    print("✅ Análisis completado exitosamente")
    return imagen, espacios

def pipeline_dataset_completo(carpeta_imagenes, target_size=(128, 128)):
    """
    PLANTILLA 2: Pipeline completo para procesamiento de dataset de imágenes.
    Desde carga masiva hasta modelo entrenado y evaluado.

    Args:
        carpeta_imagenes (str): Ruta a carpeta con subcarpetas por categoría
        target_size (tuple): Tamaño objetivo para todas las imágenes

    Returns:
        dict: Diccionario completo con imágenes, características, datos ML y modelo

    Estructura esperada de carpetas:
        carpeta_imagenes/
        ├── categoria1/
        │   ├── imagen1.jpg
        │   └── imagen2.jpg
        └── categoria2/
            ├── imagen3.jpg
            └── imagen4.jpg
    """
    print("🔄 Iniciando pipeline completo de dataset...")

    imagenes = []
    etiquetas = []

    # PASO 1: Carga masiva por categorías
    print("📁 Cargando imágenes por categoría:")
    for categoria in os.listdir(carpeta_imagenes):
        categoria_path = os.path.join(carpeta_imagenes, categoria)

        # Verificar que sea directorio
        if os.path.isdir(categoria_path):
            print(f"   🔍 Procesando categoría '{categoria}'...")
            contador_categoria = 0

            # Procesar cada imagen en la categoría
            for archivo in os.listdir(categoria_path):
                # Verificar extensiones de imagen válidas
                if archivo.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
                    path_completo = os.path.join(categoria_path, archivo)
                    img = cargar_imagen_segura(path_completo, target_size)

                    if img is not None:
                        imagenes.append(img)
                        etiquetas.append(categoria)
                        contador_categoria += 1

            print(f"      ✓ {contador_categoria} imágenes cargadas")

    total_imagenes = len(imagenes)
    print(f"\n📊 Total: {total_imagenes} imágenes cargadas desde {len(set(etiquetas))} categorías")

    if total_imagenes == 0:
        print("❌ Error: No se encontraron imágenes válidas")
        return None

    # PASO 2: Conversión a arrays NumPy
    X = np.array(imagenes)
    y = np.array(etiquetas)

    # PASO 3: Extracción masiva de características
    print(f"\n🎯 Extrayendo características de {total_imagenes} imágenes...")
    features = []

    # Barra de progreso simulada
    for i, img in enumerate(imagenes):
        feat = extraer_features_completas(img)
        features.append(feat)

        # Mostrar progreso cada 10%
        if (i + 1) % max(1, total_imagenes // 10) == 0:
            progreso = (i + 1) / total_imagenes * 100
            print(f"      📈 Progreso: {progreso:.0f}% ({i + 1}/{total_imagenes})")

    X_features = np.array(features)
    print(f"✓ Características extraídas: {X_features.shape[1]} features por imagen")

    # PASO 4: Preparación para ML
    print(f"\n🤖 Preparando datos para Machine Learning...")
    datos_ml = preparar_datos_ml(X_features, y)

    # PASO 5: Entrenamiento y evaluación rápida
    print(f"\n🎲 Entrenamiento de modelo de validación...")
    modelo, pred_val, pred_test = entrenar_modelo_rapido(datos_ml)

    # PASO 6: Compilar resultados
    resultado = {
        'imagenes_originales': X,           # Arrays de imágenes
        'etiquetas': y,                     # Etiquetas de categorías
        'caracteristicas': X_features,      # Features extraídos
        'datos_ml': datos_ml,              # Conjuntos train/val/test preparados
        'modelo': modelo,                   # Modelo entrenado
        'predicciones_val': pred_val,      # Predicciones en validación
        'predicciones_test': pred_test     # Predicciones en test
    }

    print(f"\n🎉 Pipeline completado exitosamente!")
    print(f"   📝 Datos listos para análisis avanzado")

    return resultado

def segmentacion_rapida(imagen, metodo='kmeans', **kwargs):
    """
    PLANTILLA 3: Segmentación rápida con múltiples algoritmos disponibles.
    Permite experimentar fácilmente con diferentes técnicas de segmentación.

    Args:
        imagen (numpy.ndarray): Imagen a segmentar
        metodo (str): Método de segmentación ('kmeans' o 'color_range')
        **kwargs: Parámetros específicos del método

    Returns:
        tuple: (resultado_segmentacion, modelo_o_parametros)

    Métodos disponibles:
        - 'kmeans': Clustering automático en espacio HSV
          Parámetros: n_clusters=5
        - 'color_range': Segmentación por rango de color HSV
          Parámetros: h_min=0.0, h_max=0.1, s_min=0.2, v_min=0.2
    """
    print(f"🎯 Iniciando segmentación usando método '{metodo}'...")

    if metodo == 'kmeans':
        # SEGMENTACIÓN POR CLUSTERING
        n_clusters = kwargs.get('n_clusters', 5)
        print(f"   🔬 Aplicando K-means con {n_clusters} clusters...")

        segmentacion, modelo = segmentar_por_color_hsv_auto(imagen, n_clusters)
        parametros = {'metodo': 'kmeans', 'n_clusters': n_clusters}

    elif metodo == 'color_range':
        # SEGMENTACIÓN POR RANGO DE COLOR
        h_min = kwargs.get('h_min', 0.0)
        h_max = kwargs.get('h_max', 0.1)
        s_min = kwargs.get('s_min', 0.2)
        v_min = kwargs.get('v_min', 0.2)

        print(f"   🌈 Segmentando rango HSV: H[{h_min:.2f}-{h_max:.2f}], S≥{s_min:.2f}, V≥{v_min:.2f}")

        imagen_hsv = color.rgb2hsv(imagen)
        segmentacion = crear_mascara_color_rango(imagen_hsv, h_min, h_max, s_min, v_min)

        modelo = None
        parametros = {
            'metodo': 'color_range',
            'h_min': h_min, 'h_max': h_max,
            's_min': s_min, 'v_min': v_min
        }

    else:
        print(f"❌ Error: Método '{metodo}' no reconocido")
        print("   💡 Métodos disponibles: 'kmeans', 'color_range'")
        return None, None

    # Mostrar resultados comparativos
    print("   📊 Generando visualización comparativa...")
    titulo_resultado = f"Segmentación ({metodo.upper()})"
    mostrar_antes_despues(imagen, segmentacion, "Original", titulo_resultado)

    # Estadísticas de la segmentación
    if metodo == 'kmeans':
        regiones_unicas = len(np.unique(segmentacion))
        print(f"   ✓ {regiones_unicas} regiones identificadas")
    else:
        pixeles_segmentados = np.sum(segmentacion)
        total_pixeles = segmentacion.size
        porcentaje = (pixeles_segmentados / total_pixeles) * 100
        print(f"   ✓ {pixeles_segmentados} píxeles segmentados ({porcentaje:.1f}% del total)")

    print("🎯 Segmentación completada")
    return segmentacion, {'modelo': modelo, 'parametros': parametros}

# Mensajes informativos para el usuario
print("✓ Plantillas rápidas definidas y documentadas")
print("\n🚀 Plantillas disponibles:")
print("   1️⃣  analisis_rapido_imagen(path) - Análisis integral de imagen individual")
print("   2️⃣  pipeline_dataset_completo(carpeta) - Procesamiento completo de dataset")
print("   3️⃣  segmentacion_rapida(imagen, metodo, **params) - Segmentación experimental")
print("\n💡 Cada plantilla incluye mensajes de progreso y manejo de errores")

## EJEMPLOS DE USO

### Ejemplo 1: Análisis rápido de imagen
```python
# Análisis completo con una sola función
imagen, espacios = analisis_rapido_imagen('mi_imagen.jpg')

# Acceder a diferentes espacios de color
img_hsv = espacios['hsv']
img_gray = espacios['gray']
```

### Ejemplo 2: Segmentación por color
```python
# Segmentación automática con K-means
seg, info = segmentacion_rapida(imagen, 'kmeans', n_clusters=4)

# Segmentación por rango de color específico (rojos)
seg, info = segmentacion_rapida(imagen, 'color_range',
                               h_min=0.9, h_max=1.0,
                               s_min=0.5, v_min=0.3)
```

### Ejemplo 3: Pipeline completo de dataset
```python
# Procesamiento automático de dataset completo
resultado = pipeline_dataset_completo('mi_dataset/')

# Acceder a componentes individuales
modelo = resultado['modelo']
datos_ml = resultado['datos_ml']
caracteristicas = resultado['caracteristicas']

# Análisis avanzado
mostrar_matriz_confusion(datos_ml['y_test'],
                        resultado['predicciones_test'])
```

### Ejemplo 4: Extracción manual de características
```python
# Cargar y preparar imagen
img = cargar_imagen_segura('imagen.jpg', target_size=(128, 128))

# Extraer diferentes tipos de características
features_completas = extraer_features_completas(img)  # 46 características
features_color = extraer_features_color_completas(img)  # 15 características
features_textura = extraer_features_textura_avanzadas(img)  # 7 características

print(f"Características extraídas: {len(features_completas)} total")
```

### Ejemplo 5: Visualización avanzada
```python
# Grid de múltiples imágenes
imagenes = [img1, img2, img3, img4]
titulos = ['Original', 'Filtrada', 'Segmentada', 'Resultado']
crear_grid_imagenes(imagenes, titulos, filas=2)

# Dashboard completo
crear_dashboard_imagen(imagen,
                      mostrar_canales=True,
                      mostrar_histograma=True,
                      mostrar_estadisticas=True)
```

---

## ¡Utilidades listas para usar!

**🎯 Para usar en laboratorios:**
1. Ejecuta la celda de **importaciones** al inicio
2. Ejecuta las celdas de **utilidades** que necesites
3. Usa las **plantillas rápidas** para tareas comunes
4. Consulta el **glosario** para términos técnicos

**💡 Consejos:**
- Todas las funciones incluyen documentación detallada
- Los mensajes de progreso te mantienen informado
- Manejo robusto de errores en todas las operaciones
- Parámetros por defecto optimizados para la mayoría de casos

---

# GLOSARIO TÉCNICO

## Términos de Procesamiento de Imágenes

**Array/Arreglo**: Estructura de datos multidimensional de NumPy. Para imágenes RGB: shape (altura, ancho, 3)

**BGR vs RGB**: Orden de canales de color. OpenCV usa BGR (Azul-Verde-Rojo), PIL/matplotlib usan RGB (Rojo-Verde-Azul)

**Bounding Box**: Rectángulo mínimo que contiene completamente un objeto detectado

**Canal**: Una de las componentes de color en una imagen (ej: R, G, B en RGB o H, S, V en HSV)

**Clustering**: Técnica de agrupamiento que divide datos en grupos similares sin supervisión

**Componentes Conectados**: Regiones de píxeles conectados que comparten características similares

**Elemento Estructurante/Kernel**: Matriz pequeña usada en operaciones morfológicas (ej: círculo, cuadrado)

**Escala de Grises**: Imagen con un solo canal representando intensidad de luminancia (0=negro, 1=blanco)

**Espacio de Color**: Sistema de coordenadas para representar colores (RGB, HSV, LAB, etc.)

**Máscara Binaria**: Imagen de 2 valores (0 y 1) que indica qué píxeles pertenecen a una región de interés

**Normalización**: Proceso de escalar valores a un rango estándar (ej: [0,255] → [0,1])

**Píxel**: Unidad mínima de una imagen digital. Contiene valores de intensidad para cada canal de color

**Segmentación**: Proceso de dividir una imagen en regiones homogéneas o objetos de interés

**Umbralización/Thresholding**: Técnica que convierte imagen en escala de grises a binaria usando un valor umbral

## Espacios de Color

**RGB (Red-Green-Blue)**:
- R: Intensidad de rojo [0,1]
- G: Intensidad de verde [0,1]  
- B: Intensidad de azul [0,1]
- Aditivo (suma colores para crear blanco)

**HSV (Hue-Saturation-Value)**:
- H: Tono/Matiz [0,1] (tipo de color: rojo≈0, verde≈0.33, azul≈0.67)
- S: Saturación [0,1] (pureza del color: 0=gris, 1=color puro)
- V: Valor/Brillo [0,1] (intensidad luminosa: 0=negro, 1=brillante)

**LAB (L*a*b*)**:
- L: Luminancia [0,100] (brillo perceptual)
- a: Verde (-) a Magenta (+)
- b: Azul (-) a Amarillo (+)
- Perceptualmente uniforme (diferencias numéricas = diferencias visuales)

## Operaciones Morfológicas

**Erosión**: Reduce el tamaño de regiones blancas (adelgaza objetos)

**Dilatación**: Expande el tamaño de regiones blancas (engrosa objetos)

**Apertura (Opening)**: Erosión seguida de dilatación (elimina ruido pequeño)

**Cierre (Closing)**: Dilatación seguida de erosión (cierra huecos pequeños)

## Filtros y Detección de Bordes

**Filtro Sobel**: Detecta gradientes (cambios de intensidad) en direcciones horizontal y vertical

**Detector Canny**: Algoritmo avanzado de detección de bordes con supresión de no-máximos

**Gradiente**: Medida del cambio de intensidad entre píxeles vecinos

## Características (Features) para ML

**Estadísticas de Primer Orden**: Media, desviación estándar, mediana (distribución de intensidades)

**Estadísticas de Segundo Orden**: Correlación, contraste, homogeneidad (relaciones espaciales)

**Histograma**: Distribución de frecuencias de intensidades de píxeles

**Textura**: Patrones espaciales de variación de intensidad (rugosidad, suavidad, regularidad)

## Machine Learning

**Conjunto de Entrenamiento (Train)**: Datos usados para enseñar al modelo (≈60%)

**Conjunto de Validación (Val)**: Datos para ajustar hiperparámetros y evitar overfitting (≈20%)

**Conjunto de Prueba (Test)**: Datos para evaluación final del modelo (≈20%)

**Características/Features**: Variables numéricas que describen cada muestra (ej: color promedio, textura)

**Etiquetas/Labels**: Categorías o valores objetivo que el modelo debe predecir

**Matriz de Confusión**: Tabla que muestra aciertos y errores de clasificación por cada clase

**Overfitting**: Modelo que memoriza datos de entrenamiento pero no generaliza a datos nuevos

**RandomForest**: Algoritmo que combina múltiples árboles de decisión para mayor robustez

**Validación Estratificada**: División que mantiene la proporción de cada clase en todos los conjuntos

## Métricas de Evaluación

**Accuracy (Exactitud)**: Proporción de predicciones correctas sobre el total

**Precisión**: De las predicciones positivas, cuántas fueron correctas

**Recall (Sensibilidad)**: De los casos positivos reales, cuántos fueron detectados

**F1-Score**: Media armónica entre precisión y recall

## Bibliotecas Python

**NumPy**: Operaciones numéricas y arrays multidimensionales eficientes

**PIL (Pillow)**: Carga, manipulación y guardado de imágenes en múltiples formatos

**OpenCV (cv2)**: Biblioteca completa de computer vision (¡usa formato BGR!)

**scikit-image**: Algoritmos especializados de procesamiento de imágenes

**scikit-learn**: Herramientas de machine learning y análisis de datos

**matplotlib**: Creación de gráficos y visualización de imágenes

**seaborn**: Visualizaciones estadísticas avanzadas basadas en matplotlib

---

## Referencias Rápidas de Rangos

### Rangos de Tono (H) en HSV para Colores Comunes:
- **Rojo**: 0.0-0.1 y 0.9-1.0 (está en ambos extremos)
- **Naranja**: 0.05-0.15
- **Amarillo**: 0.15-0.25  
- **Verde**: 0.25-0.45
- **Cian**: 0.45-0.55
- **Azul**: 0.55-0.75
- **Magenta**: 0.75-0.9

### Valores Típicos de Saturación (S) y Valor (V):
- **Colores vivos**: S > 0.5, V > 0.5
- **Colores pasteles**: S < 0.5, V > 0.7
- **Colores oscuros**: V < 0.3 (independiente de S)
- **Grises**: S < 0.2 (independiente de V)

### Tamaños de Kernel Recomendados:
- **Ruido pequeño**: kernel = 2-3 píxeles
- **Detalles medianos**: kernel = 5-7 píxeles  
- **Objetos grandes**: kernel = 10+ píxeles

---

## ¡Todo listo para el laboratorio integrador!

Este cuaderno te proporciona todas las herramientas necesarias para trabajar eficientemente en el procesamiento de imágenes y machine learning.